# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set notebook parameters
3. Create slab configurations, view possible terminations and select the termination pair for the interface
4. Set strain matching algorithm parameters (optional)
5. Generate interfaces with strain matcher and analyze the plot of strain vs number of atoms
6. Select the interface with the desired strain and visualize it
7. Pass the selected interface to the outside runtime

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**film**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
IS_TERMINATIONS_SELECTION_INTERACTIVE = False # Enable interactive selection of terminations via UI prompt

## 1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[0]
film = materials[1]

### 1.4. Preview Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs and select termination pair

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration

film_slab_configuration = SlabConfiguration(
    bulk=film,
    # the Miller indices of the interfacial plane of the layer
    miller_indices=(0, 0, 1),
    thickness=1,
    vacuum=1,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    # the Miller indices of the interfacial plane of the substrate
    miller_indices=(0,0,1),
    thickness=3,
    vacuum=3,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

film_slab_terminations = film_slab_configuration.terminations
substrate_slab_terminations = substrate_slab_configuration.terminations

print(f"Film terminations: {film_slab_terminations}")
print(f"Substrate terminations: {substrate_slab_terminations}")

### 2.2. Visualize slabs for possible terminations

In [ ]:
for termination in film_slab_terminations:
    visualize(film_slab_configuration.get_slab(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")

for termination in substrate_slab_terminations:
    visualize(substrate_slab_configuration.get_slab(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")    

### 2.3. Select the termination pair for the interface

In [ ]:
from itertools import product
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

# Set the termination pair indices
SUBSTRATE_TERMINATION_INDEX = 0
FILM_TERMINATION_INDEX = 0

termination_pair = (film_slab_terminations[FILM_TERMINATION_INDEX], substrate_slab_terminations[SUBSTRATE_TERMINATION_INDEX])
termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs, element_name="film/substrate termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs, element_name="film/substrate termination pair")

## 3. Create interfaces

### 3.1. Initialize the Interface Configuration

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

film_termination, substrate_termination = termination_pair
interface_configuration = InterfaceConfiguration(film_slab_configuration, substrate_slab_configuration, film_termination, substrate_termination, distance_z=3.0, vacuum=5.0)

### 3.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters
zsl_strain_matching_parameters = ZSLStrainMatchingParameters(max_area = 100)

### 3.3. Generate interfaces with strain matcher
Interfaces are sorted by size and strain.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingInterfaceBuilder, ZSLStrainMatchingInterfaceBuilderParameters

matched_interfaces_builder = ZSLStrainMatchingInterfaceBuilder(build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(strain_matching_parameters=zsl_strain_matching_parameters))

interfaces_sorted_by_size_and_strain= matched_interfaces_builder.get_materials(configuration=interface_configuration)

## 4. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interfaces_sorted_by_size_and_strain, PLOT_SETTINGS)

## 5. Select the interface to pass outside this kernel

### 5.1. Select the interface with the desired strain

Select the index for the interface with the lowest strain and the smallest number of atoms.

In [ ]:
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = slice(0, 1)
selected_interfaces = interfaces_sorted_by_size_and_strain[interfaces_slice_range_or_index]

### 5.2. Visualize the selected interface(s)

In [ ]:
visualize(selected_interfaces, repetitions=[1, 1, 1], rotation="0x")

### 6. Pass data to the outside runtime
Enrich the selected interfaces names with the strain values and pass them to the application runtime.

In [ ]:
from utils.jupyterlite import set_data

for interface in selected_interfaces:
    if "Interface, Strain:" not in interface.name:
        interface.name = f'{interface.name}, Interface, Strain:{interface.metadata["mean_abs_strain"] * 100:.3f}%'

materials_as_json = [selected_interface.to_json() for selected_interface in selected_interfaces]
set_data("materials", materials_as_json)